In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
proj_path = '/content/drive/MyDrive/Model Development '
os.chdir(proj_path)
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1_QHmZZAWiu1Idc_qP56o_8AWHyZMtSIR/Model Development '

In [3]:
#Imports
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Concatenate
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
!pip install -q pydot
!pip install graphviz
!pip install pydotplus
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.datasets import cifar10
from keras.engine import training
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average
from keras.losses import categorical_crossentropy
from keras.models import Model, Input
from keras.optimizers import Adam
from tensorflow.python.framework.ops import Tensor
from typing import Tuple, List
import glob
import numpy as np
import os

In [5]:
#image sizing
IMAGE_SIZE=[224,224,3]
train_path='/content/drive/MyDrive/Model Development /tbdataset/Train'
valid_path='/content/drive/MyDrive/Model Development /tbdataset/Test'

In [6]:
# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Preprocessing the Train set
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Model Development /tbdataset/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Model Development /tbdataset/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                            shuffle=False)

Found 3167 images belonging to 2 classes.
Found 791 images belonging to 2 classes.


In [7]:
vgg19 = tf.keras.models.load_model('saved_models/vgg19')
for layer in vgg19.layers:
    layer._name = layer._name + str('_19')

In [8]:
effnet = tf.keras.models.load_model('saved_models/efficient')
for layer in effnet.layers:
    layer._name = layer._name + str('_B1')

In [9]:
densenet = tf.keras.models.load_model('saved_models/densenet')
for layer in densenet.layers:
    layer._name = layer._name + str('_Den')

In [10]:
from keras.layers import Input, Flatten, Concatenate, Dense, Average, Dropout
inp = Input(IMAGE_SIZE)

In [11]:
models = [vgg19, effnet, densenet]

In [16]:
def ensemble(models: List [training.Model], model_input: Tensor) -> training.Model:
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model([vgg19.inputs, effnet.inputs, densenet.inputs], y, name='ensemble')
    
    return model

In [17]:
ensemble_model = ensemble(models, inp)

In [18]:
ensemble_model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0005,
    name="Adam"),
  metrics=['accuracy',
           'AUC',
           'Precision',
           'Recall',
  ]
)
ensemble_model.summary()

Model: "ensemble"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Total params: 33,687,305
Trainable params: 50,178
Non-trainable params: 33,637,127
__________________________________________________________________________________________________


In [19]:
r = ensemble_model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20


ValueError: ignored

In [ ]:
model.save('saved_models/ensemble') 

AssertionError: ignored

In [ ]:
test_set.reset()

In [ ]:
predictions = model.predict(test_set, steps = test_set.n // 31, verbose=1)

25/25 [==============================] - 48s 1s/step


In [ ]:
y_classes = predictions.argmax(axis=1)

In [ ]:
y_classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

print('Confusion Matrix:')
print(confusion_matrix(test_set.classes, y_classes))

print("Accuracy : " ,accuracy_score(test_set.classes, y_classes))
print("Precision : " ,precision_score(test_set.classes, y_classes))
print("Recall : " ,recall_score(test_set.classes, y_classes))
print("F1 Score : " ,f1_score(test_set.classes, y_classes))

Confusion Matrix:
[[394   3]
 [ 89 305]]
Accuracy :  0.8836915297092288
Precision :  0.9902597402597403
Recall :  0.7741116751269036
F1 Score :  0.8689458689458689


In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, threshold = roc_curve(test_set.classes, y_classes)
print("SVM Area under curve -> ",auc(fpr, tpr))

SVM Area under curve ->  0.8832775000319657
